Equations of motion for a simplified model of the paper of Campbell 1943:
 - three blades represented by point masses (no hub radius)
 - three DOF for the edweside motion of the blade 
 - two DOFs for the motion of the hub in the rotor plane


The rotor is spining with the azimuthal position psi=Omega t. 
Each blade has an edgwise motion of angle theta, such that the angle of a blade at a given time is psi_i+theta_i
We will linearize the equation about theta_i=0


There are three main "options"
- which plane is the rotor in: XY, or YZ (OpenFAST)
- what is the angle convention (positive or negative about z for the XY plane,  positive about x for the YZ plane)
- the order of the blade: "increasing" (psi2=psi1+2pi/3, psi3=psi1+4pi/3) or "decreasing" (psi2=psi1-2pi/3, psi3=psi1-4pi/3). 


Front view, when plane='YZ' and order is decreasing
```
      ^ z
      |        |  /bld1
      |        |~/        angle = psi + theta1
      |        |/
y     |        B
<---- O      /  \
         bl2    bld3
```
Front view, when plane='XYneg' (angle counted from y axis) ("Ronnie") and order is decreasing
```
      ^ y
      |       |  /bld1
      |       |~/
      |       |/
      |       B
      O -----/-\---> x
          bl2   bld3
```
Front view, when plane='XYpos' (angle counted from x axis) and order is increasing
```
      ^ y
      |          /bld1
      |         /
      |        /
      |       B ) psi
      O -----/-\-------> x
          bld3  bld2
```

In [ ]:
# IPython and standard packages
import numpy as np
import matplotlib.pyplot as plt
from sympy.physics.vector import init_vprinting
from IPython.core.interactiveshell import InteractiveShell
from IPython.display import display, HTML
display(HTML("<style>.container { width:98% !important; }</style>"))
InteractiveShell.ast_node_interactivity = "all"
get_ipython().run_line_magic('matplotlib', 'inline')
get_ipython().run_line_magic('load_ext', 'autoreload')
get_ipython().run_line_magic('autoreload', '2')
init_vprinting(use_latex='mathjax', pretty_print=False)
%matplotlib inline
# Sympy
import sympy as sp
from sympy import symbols, Matrix, eye, zeros, N, sin, cos
from sympy.physics.mechanics import Point, ReferenceFrame, inertia, dynamicsymbols
# Yams sympy
from welib.yams.yams_sympy       import YAMSRigidBody, YAMSInertialBody, YAMSFlexibleBody
from welib.yams.yams_sympy_model import YAMSModel
# MBC
from welib.system.examples_stab.model5DOFs import systemMatrices

# Derive Equations of motion

In [ ]:
m, M, l, gravity = symbols('m, M, l, g')
kb, kx, ky, kz = symbols('k_b, k_x, k_y, k_z')
Omega = symbols('Omega')
time = dynamicsymbols._t
x, y, z    = dynamicsymbols('x, y, z')
xd, yd, zd = dynamicsymbols('xd, yd, zd')
psi, theta1, theta2, theta3 = dynamicsymbols('psi, theta_1, theta_2, theta_3')
psid, theta1d, theta2d, theta3d = dynamicsymbols('psid, theta1d, theta2d, theta3d')
psi1, psi2, psi3 = dynamicsymbols('psi_1, psi_2, psi_3')

In [ ]:
def simpCampModel(order='decreasing', plane='YZ'):
    # --- Angles
    psis0 = [psi + theta1, psi + theta2, psi+theta3]
    if order=='increasing':
        psis = [p0 + i*2*sp.pi/3 for i,p0 in enumerate(psis0)]
    else:
        psis = [p0 - i*2*sp.pi/3 for i,p0 in enumerate(psis0)]
    psis
    # --- Bodies
    ref = YAMSInertialBody('E') 
    if plane=='YZ':
        rel_pos = (0,y,z)
        bld1 = YAMSRigidBody('A', mass=m, rho_G=(0,0,l), J_G=(0,0,0)) 
        bld2 = YAMSRigidBody('B', mass=m, rho_G=(0,0,l), J_G=(0,0,0)) 
        bld3 = YAMSRigidBody('C', mass=m, rho_G=(0,0,l), J_G=(0,0,0)) 
    elif plane=='XYneg':
        rel_pos = (x,y,0)
        bld1 = YAMSRigidBody('A', mass=m, rho_G=(0,l,0), J_G=(0,0,0)) 
        bld2 = YAMSRigidBody('B', mass=m, rho_G=(0,l,0), J_G=(0,0,0))     
        bld3 = YAMSRigidBody('C', mass=m, rho_G=(0,l,0), J_G=(0,0,0))         
    elif plane=='XYpos':
        rel_pos = (x,y,0)
        bld1 = YAMSRigidBody('A', mass=m, rho_G=(l,0,0), J_G=(0,0,0)) 
        bld2 = YAMSRigidBody('B', mass=m, rho_G=(l,0,0), J_G=(0,0,0))     
        bld3 = YAMSRigidBody('C', mass=m, rho_G=(l,0,0), J_G=(0,0,0))     
    else:
        raise NotImplementedError('plane',plane)

    nac = YAMSRigidBody('N', mass=M, rho_G=(0,0,0), J_G=(0,0,0)) 
    bodies = [nac, bld1, bld2, bld3]
    # --- Connections
    ref.connectTo(nac, type='Free' , rel_pos=rel_pos)
    if plane=='YZ':
        axis = ref.frame.x
    elif plane=='XYneg':
        axis = -ref.frame.z
    elif plane=='XYpos':
        axis = ref.frame.z
    nac.connectTo(bld1, type='Joint', rel_pos=(0,0,0), rot_type='Axis', rot_amounts=(psis[0],axis))
    nac.connectTo(bld2, type='Joint', rel_pos=(0,0,0), rot_type='Axis', rot_amounts=(psis[1],axis))
    nac.connectTo(bld3, type='Joint', rel_pos=(0,0,0), rot_type='Axis', rot_amounts=(psis[2],axis)) 
    # --- DOFs and kinetic equations
    if plane=='YZ':
        coordinates = [theta1, theta2, theta3, y, z] #twrA.q
        speeds      = [theta1d, theta2d, theta3d, yd, zd] #twrA.qd
    else:
        coordinates = [theta1, theta2, theta3, x, y] #twrA.q
        speeds      = [theta1d, theta2d, theta3d, xd, yd] #twrA.qd
    kdeqsSubs = []
    for q,qd in zip(coordinates, speeds):
        kdeqsSubs += [(qd, sp.diff(q, time))]
    # --- Loads
    body_loads = []
    for bdy in bodies:
        if plane=='YZ':
            body_loads  += [(bdy, (bdy.masscenter, -bdy.mass * gravity * ref.frame.z))]  
        else:
            body_loads  += [(bdy, (bdy.masscenter, -bdy.mass * gravity * ref.frame.y))]  
    body_loads += [(nac, (nac.origin,  -kx * x * ref.frame.x - ky * y * ref.frame.y - kz * z * ref.frame.z ))]
    body_loads += [(bld1, (bld1.frame,  -kb * theta1 * axis ))]
    body_loads += [(bld2, (bld2.frame,  -kb * theta2 * axis ))]
    body_loads += [(bld3, (bld3.frame,  -kb * theta3 * axis ))]
    # --- Model and EOM
    model = YAMSModel(ref=ref, bodies=bodies, body_loads=body_loads, coordinates=coordinates, speeds=speeds, kdeqsSubs=kdeqsSubs)
    extraSubs=[]
    extraSubs+=[(sp.diff(psi,time), Omega)]
    if order == "decreasing":
        extraSubs+=[ (sin(psi+theta2+sp.pi/3)  , -sin(psi2+theta2) )   ]
        extraSubs+=[ (cos(psi+theta2+sp.pi/3)  , -cos(psi2+theta2) )   ]
        extraSubs+=[ (cos(psi+theta3+sp.pi/6)  ,  sin(psi3+theta3) )   ]
        extraSubs+=[ (sin(psi+theta3+sp.pi/6)  , -cos(psi3+theta3) )   ]
    elif order == "increasing":
        extraSubs+=[ (sin(psi+theta3+sp.pi/3)  , -sin(psi3+theta3) )   ]
        extraSubs+=[ (cos(psi+theta3+sp.pi/3)  , -cos(psi3+theta3) )   ]
        extraSubs+=[ (cos(psi+theta2+sp.pi/6)  ,  sin(psi2+theta2) )   ]
        extraSubs+=[ (sin(psi+theta2+sp.pi/6)  , -cos(psi2+theta2) )   ]
    extraSubs+=[ (psi, psi1 )]
    EOM=model.to_EOM(extraSubs=extraSubs)
    EOM.mass_forcing_form()
    EOM.M
    #EOM.F
    #EOM.linearize([(theta1,0),noVel=True, simplify=True)
    EOMsa=EOM.smallAngleApprox([theta1,theta2,theta3])
    M0,C0,K0,B0=EOMsa.linearize([(theta1,0),(theta2,0),(theta3,0)],noVel=True, simplify=True)

    return model, EOM, M0, C0, K0

In [ ]:
#model, EOM, M0, C0, K0 = simpCampModel(order='decreasing', plane='YZ') # "OpenFAST"
#model, EOM, M0, C0, K0 = simpCampModel(order='decreasing', plane='XYneg') # "Ronnie"
model, EOM, M0, C0, K0 = simpCampModel(order='increasing', plane='XYpos') # "Morten"
print('Linearized mass matrix')
M0
print('Linearized Damping matrix')
C0
print('Linearized Stiffness matrix')
K0    

# Derivation of replacements for increasing and decreasing blade azimuths

Unfortunatley, sympy replaces the angles, simplifying the sine and cosine automatically, so we have to do some further replacements

In [ ]:
# -- Increasing order
# psi2=psi1+2pi/3 , psi3=psi1+4pi/3
sp.sin(psi+2*sp.pi/3) # sin(psi2) for XYpos
sp.cos(psi+2*sp.pi/3) # cos(psi2) for XYpos
sp.sin(psi+4*sp.pi/3) # sin(psi3) for XYpos
sp.cos(psi+4*sp.pi/3) # cos(psi3) for XYpos

In [ ]:
# -- Decreasing order
# psi2=psi1-2pi/3 , psi3=psi1+2pi/3
sp.sin(psi+4*sp.pi/3) # sin(psi2) for XY
sp.cos(psi+4*sp.pi/3) # cos(psi2) for XY
sp.sin(psi+2*sp.pi/3) # sin(psi3) for XY
sp.cos(psi+2*sp.pi/3) # cos(psi3) for XY


# Compare with python package 

In [ ]:
model, EOM, M0, C0, K0 = simpCampModel(order='decreasing', plane='YZ') # "OpenFAST"
MM, CC, KK = systemMatrices(M, m, l, kb, ky, kz, Omega, psi1, psi2, psi3, gravity, symb=True, plane='YZ')

#model, EOM, M0, C0, K0 = simpCampModel(order='decreasing', plane='XYneg') # "Ronnie"
#MM, CC, KK = systemMatrices(M, m, l, kb, kx, ky, Omega, psi1, psi2, psi3, gravity, symb=True, plane='XYneg')

#model, EOM, M0, C0, K0 = simpCampModel(order='increasing', plane='XYpos') # "Morten"
#MM, CC, KK = systemMatrices(M, m, l, kb, kx, ky, Omega, psi1, psi2, psi3, gravity, symb=True, plane='XYpos')

om0 = sp.symbols('omega_0')
extraSubs=[(kb, om0**2 * m * l**2)]
MM-M0
KK-K0
CC-C0
